In [7]:
%pip install sentence_transformers
%pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 8.0 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sentence_transformers import SentenceTransformer
import torch
import faiss
import numpy as np

# Check if MPS is available and set the device accordingly
device = torch.device("cpu")

# Model pentru embeddings
model = SentenceTransformer('all-mpnet-base-v2', device=device)
# model = SentenceTransformer('fine_tuned_model')

# Text căutat
# query = "Impactul schimbărilor climatice asupra biodiversității."
#
# # Ground truth (listează segmentele documentului)
# documents = [
#     "Schimbările climatice cauzează o pierdere masivă a biodiversității la nivel global.",
#     "Tehnologiile verzi pot ajuta la reducerea emisiilor de CO2.",
#     "Biodiversitatea este crucială pentru ecosistemele sănătoase."
# ]

query = "Donald Trump recently visited the UK."
documents = [
    "The President of the United States visited the UK.",
    "The UK was visited by the President of the United States.",
    "ONU congress decides to ban nuclear weapons.",
    "The sky is blue.",
    "The sun is shining.",
    "Recipe for a delicious cake."
]

# Generare embeddings pentru ground truth
doc_embeddings = model.encode(documents)

# Generare embeddings
query_embedding = model.encode([query])

# Normalizează embeddings
doc_embeddings = doc_embeddings / np.linalg.norm(doc_embeddings, axis=1, keepdims=True)
query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

# Creare index FAISS
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(np.array(doc_embeddings))

# Căutare în index
k = len(documents)
distances, indices = index.search(np.array(query_embedding), k)

# Afișare rezultate
for i, idx in enumerate(indices[0]):
    fragment = documents[idx]
    score = distances[0][i] 
    print(f"Fragment: {fragment}\nGrad de similitudine: {score:.2f}")


Fragment: The President of the United States visited the UK.
Grad de similitudine: 0.79
Fragment: The UK was visited by the President of the United States.
Grad de similitudine: 0.70
Fragment: The sun is shining.
Grad de similitudine: 0.10
Fragment: The sky is blue.
Grad de similitudine: 0.09
Fragment: ONU congress decides to ban nuclear weapons.
Grad de similitudine: 0.06
Fragment: Recipe for a delicious cake.
Grad de similitudine: 0.02
